## 3. LSH for cosine similarity using random hyperplanes  
1. Shigling to get the feature matrix.  
2. Generate random hyperplanes, if two vectors **a** and **b** are on the same side of a hyperplane **H**, their dot product with the normal vector of the hyperplane **v** will have the same sign, use sketches with reduced dimension vectors to represent documents, which can reflect similarity. 
3. Locality-sensitive hashing, similar documents will fall into the same bucket and form candidate pairs, which we need to test for similarity.

In [1]:
import pandas as pd 
import os
import numpy as np
from itertools import combinations
import scipy.spatial.distance as dist

In [2]:
import numpy as np
import hashlib

# define functions to genrate random vectors (which can be treated as the normal vector of a particular hyperplanes)
# for each document, get its dot product with each of the random vector, if the result is >= 0, return 1 as signature
# else return 0 as signature, finally, the length of a sketch for a document = number of random vectors 
def signature_bit(featureMatrix, planes):
    """
    LSH signature generation using random projection
    :parameter featureMatrix: shape(n_doc, n_features)
    :parameter planes: shape(n_features, n_signature)
    return the signature bits (sketch) for all vectors, shape(n_doc, n_signature)"""
    sigs = []
    for i in featureMatrix:
        a = [0]*planes.shape[1]
        for j in range(planes.shape[1]):
            if np.dot(i, planes[:,j]) >= 0:
                a[j] = 1
        sigs.append(a)        
    sigMatrix = np.vstack(sigs) 
    return sigMatrix

def sketch(featureMatrix, n_feature, n_sig):
    """
    Return sketch vector for all documents
    parameter n_feature: 
    parameter featureMatrix: shape is (nDocs, nFeatures)
    parameter n_sig: number of signatures for each document
    """
    ref_planes = np.random.randn(n_sig, n_feature) # randomly generate n_sig vectors, dimesnion=n_feature, values in N(0,1)
    sketches = signature_bit(featureMatrix, ref_planes.T).T # transpose the result to get (n_signature,n_doc) matrix
    return sketches



# find candidate pairs by hashing the siganitures values, 
# if in at least one of the bands, the two documents hash to the same bucket, they form a candidate pair

def LSH(sigMatrix, b, r):
    """
    map similar documents into the same hash bucket
    param signMatrix: signature matrix with shape(n_signature, n_doc)
    param b: the number of bands
    param r: the number of rows in a band
    return the hash bucket: a dictionary, key is hash value, value is column number
    """
    hashBuckets = {}
    begin, end = 0, r
    count = 0
    while end <= sigMatrix.shape[0]:
        count += 1
        for col in range(sigMatrix.shape[1]):
            h_fn = hashlib.md5()
            band = str(sigMatrix[begin: begin + r, col]) + str(count)
            h_fn.update(band.encode())
            tag = h_fn.hexdigest() # use hash value as bucket tag
            
            if tag not in hashBuckets:
                hashBuckets[tag] = [col]
            elif col not in hashBuckets[tag]:
                hashBuckets[tag].append(col)
        begin += r
        end += r
    return hashBuckets

def find_pairs(li):
    """find combination pairs of similar items in the same bucket"""
    pairs = set()
    for i in combinations(li,2):
        pairs.add(i)
    return pairs

In [3]:
feature_df = pd.read_csv('data/movie_norm_featureMatrix_sample.csv',index_col=0)
combined = pd.read_csv('data/combined_info.csv')

In [4]:
featureMatrix = np.array(feature_df)
featureMatrix.shape

(2000, 4204)

In [5]:
%%time
n_feature = featureMatrix.shape[1]       # number of features)
n_sig = 200                            # number of random vectors of signatures for each doc
sketches = sketch(featureMatrix,n_feature,n_sig)
LSH_buckets = LSH(sketches,20,10) # number of bands = 20, number of rows=10 in each band

Wall time: 9.95 s


In [6]:
len(LSH_buckets)

12923

In [7]:
candi_pairs = set()
for key, value in LSH_buckets.items():
    if len(value) >= 2:
        pairs = find_pairs(value)
        candi_pairs.update(pairs)
len(candi_pairs)

113297

In [8]:
pair_df = pd.DataFrame(sorted(candi_pairs),columns=['movie1','movie2'])

In [12]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(combined.index, index=combined['title']).drop_duplicates()

## 3.1 after having the buckets and candidate pairs of similar movies, we can  
### A) recommend most popular movies in the same bucket

In [13]:
def give_popula(m_idx):
    return feature_df.iloc[m_idx,-4]

In [15]:
pair_df['popularity'] = pair_df.apply(lambda x: give_popula(x['movie2']),axis=1)

In [17]:
pair_df_new = pair_df.set_index(['movie1','movie2'])

In [40]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_popular(title):
    """
    Given a movie title, we first find it's movieID, then get the movies that are in the same bucket of that movie,
    sort these movies based on popularity and return the top 10 with highest popularity.

    """
    idx = indices[title]
    try:
        sim_scores = pair_df_new.loc[idx,:].sort_values(by=['popularity'],ascending=False)
    except:
        print('No similar movies based on the given movie')
        print('Recommend the most popular movies')
        popular = feature_df.sort_values(by=['popularity'],ascending=False)
        movie_indices = popular.index[:10]
        recom_list = []
        for i in movie_indices:
            a = combined.loc[i,'title']
            recom_list.append(a)
        return recom_list
    
    if len(sim_scores) >=10:
        movie_indices = list(sim_scores.index[:10])
    else:
        movie_indices = list(sim_scores.index)
    recom_list = []
    for i in movie_indices:
        a = combined.loc[i,'title']
        recom_list.append(a)
    
    return recom_list

In [41]:
get_recommendations_popular('Jumanji')

['Star Wars',
 'The Godfather: Part II',
 'Raiders of the Lost Ark',
 'The Empire Strikes Back',
 'Addams Family Values',
 'Robin Hood: Men in Tights',
 'The Good, the Bad and the Ugly',
 'Field of Dreams',
 'The English Patient',
 'The Princess Bride']

### B) recommend most similar movies in the same bucket

In [21]:
# function calculate cosine similarities 
def cal_cosine(m1_idx,m2_idx):
    return 1-dist.cosine(feature_df.iloc[m1_idx,:],feature_df.iloc[m2_idx,:])

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_cosine(title):
    """
    Given a movie title, we first find it's index in the movie profile (not movieID), 
    then get the movies that are in the same bucket of that movie,
    calculate cosine similarities and sort these movies based on cosine similarities and 
    return the top 10 with highest similarities.
    If the given movie has no other item in the same bucket, return the most popular movies

    """
    idx = indices[title]
    try:
        sim_df = pair_df[pair_df['movie1']==idx].copy()
        sim_df['cosine'] = sim_df.apply(lambda x: cal_cosine(x['movie1'], x['movie2']),axis=1)
        sim_scores = sim_df['cosine'].sort_values(ascending=False)
    except:
        print('No similar movies based on the given movie')
        print('Recommend the most popular movies')
        popular = feature_df.sort_values(by=['popularity'],ascending=False)
        movie_indices = popular.index[:10]
        recom_list = []
        for i in movie_indices:
            a = combined.loc[i,'title']
            recom_list.append(a)
        return recom_list
    
    if len(sim_scores) >=10:
        movie_indices = list(sim_scores.index[:10])
    else:
        movie_indices = list(sim_scores.index)
    recom_list = []
    for i in movie_indices:
        a = combined.loc[i,'title']
        recom_list.append(a)
    
    return recom_list

In [22]:
get_recommendations_cosine('Jumanji')

['Bushwhacked',
 'Disclosure',
 'Before the Rain',
 'Living in Oblivion',
 'First Knight',
 'The Neon Bible',
 'Boys on the Side',
 'Moonlight and Valentino',
 'The Browning Version',
 'The Babysitter']